> **Note:** This notebook does not contain any raw or derived data from the Sleep Heart Health Study (SHHS).  
> All cell outputs have been removed to comply with the NSRR data use agreement.  
> To reproduce results, please obtain the dataset from [https://sleepdata.org/datasets/shhs](https://sleepdata.org/datasets/shhs).

In [ ]:
import os, textwrap
import numpy as np
import pandas as pd
from pyedflib import EdfReader
import mne
import plotly.graph_objects as go


EDF_PATH = os.path.expanduser(
    'data/nsrr/shhs/polysomnography/edfs/shhs2/shhs2-200077.edf'
)
N_SHOW = 10

def inspect_edf_full(edf_path, n_show=10):
    if not os.path.isfile(edf_path):
        raise FileNotFoundError(f"File not found: {edf_path}")

    with EdfReader(edf_path) as f:
        print(f"\n== File: {edf_path} ==")
        print(f"Signal count: {f.signals_in_file}")
        print(f"Duration:      {f.file_duration:.2f} s\n")

        labels   = f.getSignalLabels()
        sfs      = f.getSampleFrequencies()
        nsamps   = f.getNSamples()

        # channel overview
        print("Channel overview (data type & first few values)")
        print("-" * 60)
        for idx, label in enumerate(labels):
            sf        = sfs[idx]
            n_total   = nsamps[idx]
            n_read    = min(n_total, n_show)
            sig       = f.readSignal(idx, start=0, n=n_read)
            dtype     = sig.dtype

            # Meta info
            print(f"[{idx:2d}] {label:<20s} | {sf:6.1f} Hz | "
                  f"Total: {n_total:6d} | dtype: {dtype}")

            # First values
            for k, v in enumerate(sig):
                print(f"      sample[{k:2d}] = {v}")
            if n_total > n_read:
                print("      ...")
            print("-" * 60)

        # Header info
        print("\nHeader-Felder (mit Datentyp)")
        print("-" * 60)
        for key, val in f.getHeader().items():
            print(f"{key:<25s}: {repr(val):<35s} (type: {type(val).__name__})")

if __name__ == "__main__":
    inspect_edf_full(EDF_PATH, N_SHOW)


In [ ]:
raw = mne.io.read_raw_edf(EDF_PATH, preload=True)

%matplotlib inline

fig = raw.plot(
    n_channels=10,
    duration=5.0,
    scalings='auto',
    show_scrollbars=False,
    show=False
)
# fig.savefig('raw_snapshot.png', dpi=150)


In [ ]:
EDF_PATH = os.path.expanduser(
    'data/nsrr/shhs/polysomnography/edfs/shhs2/shhs2-200077.edf'
)

def edf_channel_overview(edf_path, n_first=5):
    with EdfReader(edf_path) as f:
        labels = f.getSignalLabels()
        sfs    = f.getSampleFrequencies()
        nsamps = f.getNSamples()
        dtypes = []
        firsts = []
        for idx in range(len(labels)):
            sig = f.readSignal(idx, start=0, n=min(n_first, nsamps[idx]))
            dtypes.append(sig.dtype.name)
            s = ', '.join(f"{v:.2f}" for v in sig)
            # engeres Wrapping für mehr Zeilen, weniger Breite
            wrapped = '<br>'.join(textwrap.wrap(s, width=15))
            firsts.append(wrapped)
    return pd.DataFrame({
        'Channel':        labels,
        'Sampl.Freq.':  sfs,
        'Total samples': nsamps,
        'Dtype':        dtypes,
        'First values':  firsts,
    })

df_overview = edf_channel_overview(EDF_PATH)

# Plotly Table
fig = go.Figure(data=[go.Table(
    columnwidth = [100, 100, 120, 80, 180],
    header=dict(
        values=list(df_overview.columns),
        fill_color='#343a40',
        font=dict(color='white', size=14),
        align='left'
    ),
    cells=dict(
        values=[df_overview[col] for col in df_overview.columns],
        fill_color=[['#f0f0f0']*len(df_overview)],
        align='left',
        height=30,
        font=dict(color='black', size=12),
        format=['text']*5
    )
)])

fig.update_layout(
    width=  100 + 100 + 120 + 80 + 180 + 40,
    height= 40 + 30*len(df_overview),
    margin=dict(l=10,r=10,b=10,t=10)
)

fig.show()
# fig.write_image('edf_channel_overview_narrow.png', scale=2)
